*PREGUNTA 4*

In [10]:
import pandas as pd

capsalera = [
    'order_id', 'activation_time_local', 'country_code', 'store_address', 'final_status', 'payment_status', 'products', 'products_total', 'purchase_total_price'
]
dataold = pd.read_csv('/Users/gabrielrogetdeaysa/Library/CloudStorage/GoogleDrive-rogetaysa@gmail.com/La meva unitat/IronhackGD/RepoIronhack/Project 3 - Ironhack Delivery/project_dataset/python_raw_data/fake_orders_test.csv', header= None, names= capsalera)
dataold.to_csv('/Users/gabrielrogetdeaysa/Library/CloudStorage/GoogleDrive-rogetaysa@gmail.com/La meva unitat/IronhackGD/RepoIronhack/Project 3 - Ironhack Delivery/project_dataset/python_raw_data/fake_orders_test_headers.csv',index=False)
data = pd.read_csv('/Users/gabrielrogetdeaysa/Library/CloudStorage/GoogleDrive-rogetaysa@gmail.com/La meva unitat/IronhackGD/RepoIronhack/Project 3 - Ironhack Delivery/project_dataset/python_raw_data/fake_orders_test_headers.csv')
data['activation_time_local'] = pd.to_datetime(data['activation_time_local'], errors='coerce', format='%d/%m/%Y %H:%M')
data.head()

,order_id,activation_time_local,country_code,store_address,final_status,payment_status,products,products_total,purchase_total_price
0,33557880,NaT,AR,14200,DeliveredStatus,PAID,1,4.54,8.64
1,33512615,NaT,TR,28725,DeliveredStatus,PAID,1,3.76,3.76
2,33512451,NaT,TR,28725,DeliveredStatus,PAID,1,2.86,2.86
3,33530892,NaT,ES,19777,CanceledStatus,PAID,1,12.95,0.00
4,33557765,NaT,AR,34565,DeliveredStatus,PAID,2,2.86,6.48


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60400 entries, 0 to 60399
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   order_id                  60400 non-null  int64         
 1   activation_time_local     0 non-null      datetime64[ns]
 2   country_code              60400 non-null  object        
 3   store_address             60400 non-null  int64         
 4   final_status              60400 non-null  object        
 5   payment_status            60400 non-null  object        
 6   products                  60400 non-null  int64         
 7   products_total            60400 non-null  float64       
 8   purchase_total_price      60400 non-null  float64       
 9   subautorizado             60400 non-null  bool          
 10  autorizacion_incremental  60400 non-null  bool          
 11  diferencia_preu           60400 non-null  float64       
dtypes: bool(2), dateti

In [9]:
data.describe()

,order_id,activation_time_local,store_address,products,products_total,purchase_total_price,diferencia_preu
count,6.040000e+04,0,60400.000000,60400.000000,60400.000000,60400.000000,60400.000000
mean,3.278265e+07,NaT,41580.723692,2.811358,9.832693,10.703447,0.870754
min,3.150378e+07,NaT,190.000000,1.000000,0.000000,0.000000,-221.480000
25%,3.240091e+07,NaT,21110.000000,1.000000,4.110000,4.330000,0.000000
50%,3.278110e+07,NaT,45849.000000,2.000000,7.120000,7.980000,0.310000
75%,3.318279e+07,NaT,61992.250000,3.000000,12.750000,14.160000,2.210000
max,3.356085e+07,NaT,75236.000000,39.000000,221.480000,265.760000,264.360000
std,4.453627e+05,NaN,21905.784563,2.431214,9.276841,10.781632,7.818695


### 1. ¿Qué porcentaje de pedidos están subautorizados?


In [14]:
data['subautorizado'] = data['products_total']<data['purchase_total_price'] 
porcentaje_subautorizado = data['subautorizado'].mean()*100
print(f"P1: % Subautorizados: {porcentaje_subautorizado:.2f}%")

P1: % Subautorizados: 57.58%


### 2. ¿Qué porcentaje de pedidos se autorizarían correctamente con una autorización incremental (+20%) sobre el monto en el checkout?

In [15]:
data['autorizacion_incremental']= data['products_total']*1.2 >= data['purchase_total_price']
percentatge_autoritzats_incremental = data['autorizacion_incremental'].mean()*100
print(f"P2: % de pedidos autoritzats amb increment de 20%: {percentatge_autoritzats_incremental:.2f}%")

P2: % de pedidos autoritzats amb increment de 20%: 69.03%


### 3. ¿Hay diferencias cuando se dividen por país?

In [20]:
por_pais = data.groupby('country_code')['autorizacion_incremental'].mean()*100
print("3: ")
por_pais

3: 


country_code
AR    66.388720
BR    50.000000
CI    66.666667
CL    12.952899
CR    79.855465
DO    62.000000
EC    62.599681
EG    65.645514
ES    71.497735
FR    67.322097
GE    68.315018
GT    79.717813
IT    70.085470
KE    73.404255
MA    77.512255
PA    66.929911
PE    78.098224
PR    87.878788
PT    63.916392
RO    72.173512
TR    68.887839
UA    72.663268
UY    79.473684
Name: autorizacion_incremental, dtype: float64

### 4. Para el resto de pedidos que quedarían fuera de la autorización incremental, ¿qué valores serían necesarios para capturar el monto restante?

In [22]:
sin_autorizacion_incremental = data.loc[data['subautorizado']& data['autorizacion_incremental'],'purchase_total_price']-data['products_total']
print("4: ")
display(sin_autorizacion_incremental)

4: 


0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
60395    0.16
60396     NaN
60397    0.67
60398     NaN
60399    0.44
Length: 60400, dtype: float64

In [33]:
def calcular_monto_extra(row):
    if row['subautorizado'] and not row['autorizacion_incremental']:
        if row['products_total'] == 0:
            return 10.00, row['purchase_total_price']
        else:
            pct_extra = (row['purchase_total_price'] / row['products_total']) - 1
            cantidad_extra = row['purchase_total_price'] - row['products_total']
            return pct_extra, cantidad_extra
    return 0, 0

data[['pct_monto_extra', 'cantidad_extra']] = data.apply(lambda row: pd.Series(calcular_monto_extra(row)), axis=1)

resultats = data[(data['pct_monto_extra'] != 0) | (data['cantidad_extra'] != 0)]

display(resultats[['products_total', 'purchase_total_price', 'pct_monto_extra', 'cantidad_extra']])

,products_total,purchase_total_price,pct_monto_extra,cantidad_extra
0,4.54,8.64,0.903084,4.10
4,2.86,6.48,1.265734,3.62
10,2.05,10.17,3.960976,8.12
12,8.10,10.20,0.259259,2.10
20,4.42,5.73,0.296380,1.31
...,...,...,...,...
60371,9.83,21.44,1.181078,11.61
60373,3.28,6.56,1.000000,3.28
60374,2.46,5.25,1.134146,2.79
60380,3.44,6.89,1.002907,3.45


### 5. ¿Qué tiendas son las más problemáticas en términos de pedidos y valor monetario?

In [6]:
problematicas = data.groupby('store_address').agg({'subautorizado':'sum','purchase_total_price':'sum'}).sort_values(by='subautorizado',ascending=False)
print("5: ")
print(problematicas.head())

5: 
               subautorizado  purchase_total_price
store_address                                     
28671                    448               4714.97
28712                    217               3781.39
28286                    204               1246.12
14455                    179               3514.33
11694                    160               1065.96


### 6. Para los pedidos subautorizados, ¿hay una correlación entre la diferencia en los precios y la cancelación del pedido? En otras palabras: ¿Es más probable que se cancele un pedido a medida que aumenta la diferencia de precio?

In [7]:
data['diferencia_preu']=data['purchase_total_price']-data['products_total']
correlacio = data['diferencia_preu'].corr(data['final_status'].apply(lambda x:1 if x == 'CanceledStatus' else 0))
print(f"6: Correlacio {correlacio:.2f}")

6: Correlacio -0.43
